In [3]:
import cx_Oracle
import pandas as pd
cx_Oracle.init_oracle_client(lib_dir=r"C:\Users\camendol\oracle\instantclient_19_11")

In [4]:
dsn_tns = cx_Oracle.makedsn('som-racload03.humedica.net'
                            ,'1521'
                            ,service_name='anltsts'
                            ) 

conn = cx_Oracle.connect( user=r'camendola'
                         ,password='B2sP9mzYwN3'
                         ,dsn=dsn_tns) 

In [5]:
client_hnum='H053731'
rel='202107'
iter='1'

In [6]:
inp_evt_enc_clm=f"""
select pce.*
      ,enc.*
      ,clm.*
      ,cee.client_ds_id
      ,cee.encounterid
      ,round(pce.EVT_END_DTM - pce.EVT_ADMIT_DTM,0) 
         as STAY_DAYS
      ,EXTRACT(YEAR from pce.EVT_ADMIT_DTM) 
        as evt_year
      ,case         
         when round(pce.EVT_END_DTM - pce.EVT_ADMIT_DTM,0) < 1         
           then 'Same'       
         else 'OnePlus'       
       end DAY_TYPE   
  from
    --Clinical Events
    (select * from OADW_PRD_{client_hnum}_CDR_{rel}_{iter}.L2_PAT_CLINICAL_EVENT where event_type_cui = 'CH000106') pce       
      left join
    --Event-Encounter Linking  
    (select * from OADW_PRD_{client_hnum}_CDR_{rel}_{iter}.L1_CLINICAL_EVENT_ENCOUNTER where ENCOUNTERIDTYPE_CUI = 'MASTER') cee        
      on cee.ENCOUNTER_GRP_NUM = pce.clinical_event_id   
      left join
    --Encounters  
    OADW_PRD_{client_hnum}_CDR_{rel}_{iter}.L1_CLINICALENCOUNTER enc              
      on cee.ENCOUNTERID = enc.ENCOUNTERID            
     and cee.CLIENT_DS_ID = enc.CLIENT_DS_ID
      left join
    --Claims !granularity!  
    OADW_PRD_{client_hnum}_CDR_{rel}_{iter}.L1_CLAIM clm          
      on enc.ENCOUNTERID = clm.ENCOUNTERID        
     and enc.CLIENT_DS_ID = clm.CLIENT_DS_ID  
"""

In [ ]:
test_it=f"""
select count(*) as ct from ({inp_evt_enc_clm})
"""

z = pd.read_sql(test_it, con=conn)

In [ ]:
conn.close()